In [8]:
import sys
import os

sys.path.insert(0, os.path.dirname(os.getcwd()))

In [10]:
from scripts import dataset

In [13]:
d = dataset.TrainingDataset('../training_data.bin')

In [15]:
d[0]

{'board_tensor': tensor([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [-1., -1., -1., -1., -1., -1., -1., -1.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]],
 
         [[ 0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0., -1.,  0.,  0.,  0.,  0., -1.,  0.]],
 
         [[ 0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
          [ 0.,  0.,  0.,  0.,  0.,  0.,  